## It's learning of refactored model

First, we need to define weaknesses of the approach from the article:
1. Using of ResNet50 as backbone (Some more powerfull models can be used)
2. Assumption about uniform distribution of data(num of lesions)
3. Loss function and lambda parameter

What can be done with first weakness. We can change it with more strong model like SWIN Transformer (tiny) from Microsoft. That model performed better in this task compared to ResNet50. Now, transformers are performing well not only in NLP tasks, but also in CV. However, that model provide new approach(patching). Also, we can add some self-attention layers.

Second, some new approach to multi-task learning can be applied. In article they use standard hard parametr sharing, but also soft parametr sharing can be used. 

Also, X. Sun et al. proposed, <a href="https://arxiv.org/abs/1911.12423">“AdaShare: Learning What to Share for Efficient Deep Multi-Task Learning”</a> (2020). The primary goal of the researchers was to specify a single multi-layer architecture for MTL and train a policy that determines which layers to share by multiple tasks, which layers to use for specific tasks and which layers to skip for all tasks while ensuring the model provides highest performance at its most compact form.

Second weakness is not too obvious. First, we can, that distribtion of data(from EDA) is not uniform (it's assumption, but the likehood of the value being below average is higher than the likelihood of being higher). Some other distributions can be tried (Student's t-distribution, Exponential and etc), but also we can try find by use of EM algorithm (data distribution can be mixture of gaussian).

Last question is about parametr lambda in loss function. We can try to learn that parametrs. I tried one approach from <a href="https://towardsdatascience.com/multi-task-learning-with-pytorch-and-fastai-6d10dc7ce855"> here </a> and it didn't worked well. The problem is that model starts to optimize one loss and give small weights to others.

Also we cat try to expand dataset by some simple transformations (for example by albumination). From data we know, that majority of data have low bliriness(contrast). We can change it and add new images to dataset. So, it will help us to improve stability of our model.

In [1]:
import os
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from hydra.utils import to_absolute_path

In [2]:
GlobalHydra.instance().clear()

In [3]:
initialize(config_path="./conf")

hydra.initialize()

In [4]:
cfg = compose(config_name="config.yaml")

In [5]:
cfg

{'dataset_local_train': {'data_path': '../Classification/JPEGImages', 'data_file': '../Classification/NNEW_trainval_0.txt', 'batch_size': 32, 'num_workers': 4}, 'dataset_local_test': {'data_path': '../Classification/JPEGImages', 'data_file': '../Classification/NNEW_test_0.txt', 'batch_size': 32, 'num_workers': 4}, 'optimizer': {'LR': 0.001, 'type': 'sgd'}, 'backbone': {'name': 'swin_tiny'}, 'logger': {'name': 'SGD SWIN Densed distribution 0.001', 'project': 'OpenFace'}, 'trainer': {'max_epochs': 20, 'log_every_n_steps': 3}}

In [6]:
from LDL import factory

In [7]:
def train(cfg):

    cfg['dataset_local_train']['data_path'] = to_absolute_path(cfg['dataset_local_train']['data_path'])
    cfg['dataset_local_train']['data_file'] = to_absolute_path(cfg['dataset_local_train']['data_file'])

    cfg['dataset_local_test']['data_path'] = to_absolute_path(cfg['dataset_local_test']['data_path'])
    cfg['dataset_local_test']['data_file'] = to_absolute_path(cfg['dataset_local_test']['data_file'])

    trainer, model, train_loader, test_loader = factory.get_trainer(
        cfg['backbone']['name'],
        cfg['dataset_local_train'],
        cfg['dataset_local_test'],
        cfg['logger'],
        cfg['trainer']
    )

    trainer.fit(model, train_loader, test_loader)

    return trainer, model, train_loader, test_loader

In [8]:
train(cfg)

d:\Projects\OpenFaceTest\venv\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: solidcake98 (use `wandb login --relogin` to force relogin)



  | Name              | Type                      | Params
----------------------------------------------------------------
0 | multitask_model   | MultitaskModel            | 27.6 M
1 | loss              | MultiTaskLossWrapperThird | 0     
2 | train_mae         | MeanAbsoluteError         | 0     
3 | train_accuracy    | Accuracy                  | 0     
4 | valid_mae         | MeanAbsoluteError         | 0     
5 | valid_mse         | MeanSquaredError          | 0     
6 | valid_accuracy    | Accuracy                  | 0     
7 | valid_precision   | Precision                 | 0     
8 | valid_specificity | Specificity               | 0     
9 | valid_sensetivity | Recall                    | 0     
----------------------------------------------------------------
27.6 M    Trainable params
0         Non-trainable params
27.6 M    Total params
110.290   Total estimated model params size (MB)
d:\Projects\OpenFaceTest\venv\lib\site-packages\pytorch_lightning\callbacks\model_checkpoi

Validation sanity check: 0it [00:00, ?it/s]

d:\Projects\OpenFaceTest\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(


Epoch 0:   0%|          | 0/47 [00:00<?, ?it/s] 

d:\Projects\OpenFaceTest\venv\lib\site-packages\torch\nn\functional.py:2747: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 19: 100%|██████████| 47/47 [01:08<00:00,  1.45s/it, loss=0.46, v_num=dqrp, train_loss_step=0.645, valid_loss=0.981, train_loss_epoch=0.422]


(<pytorch_lightning.trainer.trainer.Trainer at 0x23ed6cc5400>,
 LDLModel(
   (multitask_model): MultitaskModel(
     (feature_extraction): SwinTransformer(
       (patch_embed): PatchEmbed(
         (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
         (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
       )
       (pos_drop): Dropout(p=0.0, inplace=False)
       (layers): Sequential(
         (0): BasicLayer(
           dim=96, input_resolution=(56, 56), depth=2
           (blocks): ModuleList(
             (0): SwinTransformerBlock(
               (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
               (attn): WindowAttention(
                 (qkv): Linear(in_features=96, out_features=288, bias=True)
                 (attn_drop): Dropout(p=0.0, inplace=False)
                 (proj): Linear(in_features=96, out_features=96, bias=True)
                 (proj_drop): Dropout(p=0.0, inplace=False)
                 (softmax): Softmax(dim=-1)
 

Watch process of learning here https://wandb.ai/solidcake98/ImageAcneAnalysis/overview

Below I provide the picture of working of web app

<img src="./jupyter_img/TestService.png" width=1200 height=800/>